# Risk Management Dashboard - Data Cleaning & Preprocessing

This notebook handles the cleaning and preprocessing of market and macroeconomic data for the risk management dashboard. It builds on the data retrieved in the previous notebook (`01_data_retrieval.ipynb`).

## Overview of Steps

1. Load raw data from data retrieval stage
2. Inspect data quality (missing values, outliers, etc.)
3. Clean and preprocess market data
4. Clean and preprocess macroeconomic data
5. Align datasets to common dates
6. Calculate returns
7. Create portfolio based on asset weights
8. Analyze return distributions and correlations
9. Save processed data for use in risk models

In [ ]:
# Import standard libraries
import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import datetime as dt
from scipy import stats
import warnings

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 120)

# Configure plot styles
plt.style.use('seaborn-darkgrid')
sns.set_palette('muted')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

In [ ]:
# Set up project paths
PROJECT_ROOT = Path().resolve().parents[0]
CONFIG_DIR = PROJECT_ROOT / "configs"
DATA_DIR = PROJECT_ROOT / "data"
RAW_DIR = DATA_DIR / "raw"
PROCESSED_DIR = DATA_DIR / "processed"

# Make sure processed data directory exists
PROCESSED_DIR.mkdir(exist_ok=True, parents=True)

print(f"Project root: {PROJECT_ROOT}")
print(f"Raw data directory: {RAW_DIR}")
print(f"Processed data directory: {PROCESSED_DIR}")

## 1. Import Data Processing Module and Load Configuration

In [ ]:
# Add the project root to the path
import sys
sys.path.append(str(PROJECT_ROOT))

# Import data processing functions
from src.data.process_data import (
    load_data_config,
    clean_data,
    calculate_returns,
    create_portfolio,
    align_data,
    process_data_pipeline
)

# Load data processing configuration
config = load_data_config()

# Display configuration
print("Data Processing Configuration:")
for key, value in config.items():
    print(f"  - {key}: {value}")

## 2. Load Raw Data from Data Retrieval Stage

In [ ]:
# Function to get the latest file in a directory matching a pattern
def get_latest_file(directory, pattern):
    files = list(directory.glob(pattern))
    if not files:
        return None
    return max(files, key=lambda f: f.stat().st_mtime)

# Load market data - use a more specific pattern to get only Yahoo data
market_file = get_latest_file(RAW_DIR, "yahoo_data_*.csv")

# If no Yahoo data file found, try other patterns
if not market_file:
    market_file = get_latest_file(RAW_DIR, "market_data_*.csv")
    
if not market_file:
    print("No market data files found matching expected patterns.")
    print("Available files in RAW_DIR:")
    for file in RAW_DIR.glob("*.csv"):
        print(f"  - {file.name}")
    
    # Fall back to choosing from available files
    all_files = list(RAW_DIR.glob("*.csv"))
    if all_files:
        print("\nAttempting to identify market data from available files...")
        for file in all_files:
            if "fred" not in file.name.lower():  # Skip FRED files
                market_file = file
                print(f"Selected {market_file} as potential market data file")
                break

if market_file:
    print(f"Loading market data from {market_file}")
    market_data = pd.read_csv(market_file, index_col=0, parse_dates=True)
    # Check if we need to convert to MultiIndex
    if len(market_data.columns) > 0 and '_' in market_data.columns[0]:
        # This is likely a flattened MultiIndex
        try:
            market_data.columns = pd.MultiIndex.from_tuples(
                [tuple(col.split('_', 1)) for col in market_data.columns]
            )
            print("Converted columns to MultiIndex format")
        except:
            print("Could not convert columns to MultiIndex format")
    
    print(f"Market data shape: {market_data.shape}")
else:
    print("No market data files found. Please run data retrieval first.")
    market_data = pd.DataFrame()

In [ ]:
# When identifying which data is market vs. macro:
is_market_data = False
if isinstance(market_data.columns, pd.MultiIndex):
    # Check if typical market data columns exist
    is_market_data = any(col in market_data.columns.levels[0] 
                         for col in ['TRDPRC_1', 'Close', 'Adj Close'])
else:
    # Check if market tickers exist
    is_market_data = any(ticker in market_data.columns 
                        for ticker in ['SPX', 'US10YT=RR', '^GSPC', '^TNX'])

if not is_market_data:
    print("WARNING: Your market_data appears to contain macro indicators instead of market prices!")
    print("Please check your data retrieval process to ensure market data is being loaded correctly.")

In [ ]:
# After loading market_data
print("Market data columns:")
if isinstance(market_data.columns, pd.MultiIndex):
    print(f"Level 0: {market_data.columns.levels[0].tolist()}")
    print(f"Level 1: {market_data.columns.levels[1].tolist()}")
else:
    print(market_data.columns.tolist())

In [ ]:
# Load macroeconomic data
macro_file = get_latest_file(RAW_DIR, "fred_data_*.csv")

if macro_file:
    print(f"Loading macroeconomic data from {macro_file}")
    macro_data = pd.read_csv(macro_file, index_col=0, parse_dates=True)
    print(f"Macroeconomic data shape: {macro_data.shape}")
else:
    print("No macroeconomic data files found. Continuing without macro data.")
    macro_data = pd.DataFrame()

In [ ]:
if market_file:
    print(f"Loading market data from {market_file}")
    
    # First, check file format by looking at the first few lines
    with open(market_file, 'r') as f:
        first_few_lines = [f.readline() for _ in range(5)]
    
    print("File preview:")
    for line in first_few_lines:
        print(line.strip())
    
    # Try loading with proper date index
    try:
        # First row is column headers, second row has asset names
        # Date column is actually in the third row
        market_data = pd.read_csv(market_file, header=[0, 1], index_col=0, parse_dates=True)
        print("Loaded with multi-level headers")
        
        # Clean up any 'Date' in the index
        if 'Date' in market_data.index:
            market_data = market_data.drop('Date')
            print("Dropped 'Date' from index")
    except Exception as e:
        print(f"Multi-level header loading failed: {e}")
        
        try:
            # Fallback: Load with skiprows to handle the date issue
            market_data = pd.read_csv(market_file, skiprows=2, index_col=0, parse_dates=True)
            print("Loaded with skiprows=2")
        except Exception as e:
            print(f"Skiprows loading failed: {e}")
            market_data = pd.DataFrame()
    
    # Ensure all data is numeric
    for col in market_data.columns:
        try:
            market_data[col] = pd.to_numeric(market_data[col], errors='coerce')
        except:
            pass
    
    print(f"Market data shape: {market_data.shape}")
else:
    print("No market data files found. Please run data retrieval first.")
    market_data = pd.DataFrame()

## 3. Inspect Data Quality

In [ ]:
# Function to analyze data quality
def analyze_data_quality(df, title="Dataset"):
    print(f"\n=== {title} Quality Analysis ===")
    
    if df.empty:
        print("Empty DataFrame, skipping analysis")
        return
    
    # Basic info
    print(f"Shape: {df.shape}")
    
    # Check if index is datetime
    is_datetime_index = pd.api.types.is_datetime64_any_dtype(df.index)
    
    if is_datetime_index:
        # If we have a proper datetime index
        print(f"Date range: {df.index.min()} to {df.index.max()}")
        print(f"Total days: {len(df)}")
        
        # Calendar coverage
        date_range = pd.date_range(start=df.index.min(), end=df.index.max(), freq='D')
        missing_days = len(date_range) - len(df)
        print(f"Calendar days in range: {len(date_range)}")
        print(f"Missing days: {missing_days} ({missing_days / len(date_range):.2%})")
    else:
        # For non-datetime index
        print(f"Index type: {df.index.dtype}")
        print(f"Index range: {df.index.min()} to {df.index.max()}")
        print(f"Total records: {len(df)}")
        print("Note: Index is not datetime type, calendar coverage analysis skipped")
    
    # Missing values
    total_cells = df.shape[0] * df.shape[1]
    missing_cells = df.isna().sum().sum()
    print(f"Missing values: {missing_cells} ({missing_cells / total_cells:.2%})")
        
    # Missing values by column
    if isinstance(df.columns, pd.MultiIndex):
        # For MultiIndex columns, we'll check by level 0 (data type) and level 1 (asset)
        print("\nMissing values by data type:")
        for level0 in df.columns.levels[0]:
            sub_df = df[level0]
            missing = sub_df.isna().sum().sum()
            total = sub_df.shape[0] * sub_df.shape[1]
            print(f"  - {level0}: {missing} ({missing / total:.2%})")
        
        print("\nMissing values by asset (using Close/TRDPRC_1):")
        # Determine which price column to use
        price_col = None
        for col in ['TRDPRC_1', 'Close', 'Adj Close']:
            if col in df.columns.levels[0]:
                price_col = col
                break
        
        if price_col:
            for asset in df[price_col].columns:
                missing = df[price_col][asset].isna().sum()
                total = len(df)
                print(f"  - {asset}: {missing} ({missing / total:.2%})")
    else:
        print("\nMissing values by column:")
        missing_by_col = df.isna().sum()
        for col, missing in missing_by_col.items():
            if missing > 0:
                print(f"  - {col}: {missing} ({missing / len(df):.2%})")
    
    # Potential outliers
    if isinstance(df.columns, pd.MultiIndex):
        # For price data, we'll look at daily returns for outliers
        if price_col:
            print("\nPotential outliers in daily price changes:")
            for asset in df[price_col].columns:
                returns = df[price_col][asset].pct_change().dropna()
                # Identify outliers as returns beyond 3 standard deviations
                mean = returns.mean()
                std = returns.std()
                outliers = returns[abs(returns - mean) > 3 * std]
                if not outliers.empty:
                    print(f"  - {asset}: {len(outliers)} outliers ({len(outliers) / len(returns):.2%})")
                    # Show the top 5 largest outliers
                    top_outliers = outliers.abs().nlargest(5).index
                    print(f"    Top outliers: {', '.join([f'{date.date()}: {returns[date]:.2%}' for date in top_outliers])}")
    else:
        # For numeric columns, detect outliers
        numeric_cols = df.select_dtypes(include=[np.number]).columns
        if len(numeric_cols) > 0:
            print("\nPotential outliers in numeric columns:")
            for col in numeric_cols:
                values = df[col].dropna()
                # Calculate z-scores
                z_scores = np.abs((values - values.mean()) / values.std())
                outliers = z_scores[z_scores > 3]
                if len(outliers) > 0:
                    print(f"  - {col}: {len(outliers)} outliers ({len(outliers) / len(values):.2%})")

# Analyze market data quality
analyze_data_quality(market_data, "Market Data")

In [ ]:
# Analyze macroeconomic data quality
analyze_data_quality(macro_data, "Macroeconomic Data")

## 4. Visualize Raw Data

In [ ]:
# Function to plot time series data
def plot_time_series(df, title="Time Series Data", cols=None, figsize=(14, 8)):
    if df.empty:
        print(f"Empty DataFrame, skipping {title} plot")
        return
    
    # Ensure data is numeric
    numeric_df = df.copy()
    for col in numeric_df.columns:
        numeric_df[col] = pd.to_numeric(numeric_df[col], errors='coerce')
    
    plt.figure(figsize=figsize)
    
    # Handle MultiIndex DataFrame
    if isinstance(numeric_df.columns, pd.MultiIndex):
        # Determine which price column to use
        price_col = None
        for col in ['TRDPRC_1', 'Close', 'Adj Close']:
            if col in numeric_df.columns.levels[0]:
                price_col = col
                break
        
        if price_col:
            try:
                # Normalize to 100 at the start
                first_row = numeric_df[price_col].iloc[0]
                if (first_row == 0).any() or first_row.isna().any():
                    # Find first non-zero, non-NaN row
                    for i in range(1, len(numeric_df)):
                        if not (numeric_df[price_col].iloc[i] == 0).any() and not numeric_df[price_col].iloc[i].isna().any():
                            first_row = numeric_df[price_col].iloc[i]
                            break
                
                normalized = numeric_df[price_col].div(first_row) * 100
                normalized.plot(ax=plt.gca())
                plt.title(f"{title} - Normalized to 100")
                plt.ylabel("Normalized Value")
            except Exception as e:
                print(f"Error normalizing data: {e}")
                # Fallback: just plot the raw data
                numeric_df[price_col].plot(ax=plt.gca())
                plt.title(f"{title} - Raw Values")
                plt.ylabel("Value")
        else:
            print(f"No suitable price columns found in {title}")
            return
    else:
        # For regular DataFrame
        if cols is None:
            cols = numeric_df.columns
        
        try:
            # Normalize to 100 at the start
            first_row = numeric_df[cols].iloc[0]
            if (first_row == 0).any() or first_row.isna().any():
                # Find first non-zero, non-NaN row
                for i in range(1, len(numeric_df)):
                    if not (numeric_df[cols].iloc[i] == 0).any() and not numeric_df[cols].iloc[i].isna().any():
                        first_row = numeric_df[cols].iloc[i]
                        break
            
            normalized = numeric_df[cols].div(first_row) * 100
            normalized.plot(ax=plt.gca())
            plt.title(f"{title} - Normalized to 100")
            plt.ylabel("Normalized Value")
        except Exception as e:
            print(f"Error normalizing data: {e}")
            # Fallback: just plot the raw data
            numeric_df[cols].plot(ax=plt.gca())
            plt.title(f"{title} - Raw Values")
            plt.ylabel("Value")
    
    plt.xlabel("Date")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# Plot market data
plot_time_series(market_data, "Market Data")

In [ ]:
# Plot macro data
plot_time_series(macro_data, "Macroeconomic Indicators")

In [ ]:
# Visualize missing values in market data
def plot_missing_values(df, title="Missing Values"):
    if df.empty:
        print(f"Empty DataFrame, skipping {title} plot")
        return
    
    plt.figure(figsize=(12, 8))
    
    # Handle MultiIndex DataFrame
    if isinstance(df.columns, pd.MultiIndex):
        # Determine which price column to use
        price_col = None
        for col in ['TRDPRC_1', 'Close', 'Adj Close']:
            if col in df.columns.levels[0]:
                price_col = col
                break
        
        if price_col:
            # Focus on price data
            missing_data = df[price_col].isna()
            
            # Create a heatmap of missing values
            sns.heatmap(missing_data, cmap='viridis', cbar_kws={'label': 'Missing'})
            plt.title(f"Missing Values in {title} - {price_col}")
            plt.xlabel("Asset")
            plt.ylabel("Date")
        else:
            print(f"No suitable price columns found in {title}")
            return
    else:
        # For regular DataFrame
        missing_data = df.isna()
        
        # Create a heatmap of missing values
        sns.heatmap(missing_data, cmap='viridis', cbar_kws={'label': 'Missing'})
        plt.title(f"Missing Values in {title}")
        plt.xlabel("Column")
        plt.ylabel("Date")
    
    plt.tight_layout()
    plt.show()

# Plot missing values in market data
plot_missing_values(market_data, "Market Data")

## 5. Clean and Preprocess Data

In [ ]:
# Clean market data
print("Cleaning market data...")
cleaned_market = clean_data(market_data, config=config)
print(f"Original shape: {market_data.shape}, Cleaned shape: {cleaned_market.shape}")

# Clean macro data
if not macro_data.empty:
    print("\nCleaning macroeconomic data...")
    cleaned_macro = clean_data(macro_data, config=config)
    print(f"Original shape: {macro_data.shape}, Cleaned shape: {cleaned_macro.shape}")
else:
    cleaned_macro = macro_data
    print("\nSkipping macroeconomic data cleaning (no data available)")

In [ ]:
# Align market and macro data to common dates
if not cleaned_macro.empty:
    print("Aligning market and macro data to common dates...")
    aligned_market, aligned_macro = align_data(cleaned_market, cleaned_macro)
    print(f"Aligned data shapes: Market {aligned_market.shape}, Macro {aligned_macro.shape}")
else:
    aligned_market = cleaned_market
    aligned_macro = cleaned_macro
    print("Skipping data alignment (no macro data available)")

## 6. Calculate Returns

In [ ]:
# Calculate market returns
returns_method = config.get('returns_method', 'log')
print(f"Calculating {returns_method} returns for market data...")
market_returns = calculate_returns(aligned_market, method=returns_method)
print(f"Returns shape: {market_returns.shape}")

# Display return statistics
print("\nReturn statistics:")
if isinstance(market_returns.columns, pd.MultiIndex):
    # If we have MultiIndex columns, we need to handle them differently
    for asset in market_returns.columns.levels[1]:
        asset_returns = market_returns.xs(asset, axis=1, level=1)
        if not asset_returns.empty:
            print(f"  - {asset}:")
            print(f"    Mean: {asset_returns.mean().values[0]:.6f}")
            print(f"    Std Dev: {asset_returns.std().values[0]:.6f}")
            print(f"    Min: {asset_returns.min().values[0]:.6f}")
            print(f"    Max: {asset_returns.max().values[0]:.6f}")
else:
    # For regular DataFrame
    for col in market_returns.columns:
        print(f"  - {col}:")
        print(f"    Mean: {market_returns[col].mean():.6f}")
        print(f"    Std Dev: {market_returns[col].std():.6f}")
        print(f"    Min: {market_returns[col].min():.6f}")
        print(f"    Max: {market_returns[col].max():.6f}")

In [ ]:
# Visualize return distributions
def plot_return_distributions(returns, title="Return Distributions"):
    if returns.empty:
        print(f"Empty DataFrame, skipping {title} plot")
        return
    
    plt.figure(figsize=(14, 8))
    
    # Handle MultiIndex DataFrame
    if isinstance(returns.columns, pd.MultiIndex):
        for asset in returns.columns.levels[1]:
            asset_returns = returns.xs(asset, axis=1, level=1)
            if not asset_returns.empty:
                sns.kdeplot(asset_returns.iloc[:, 0], label=asset)
    else:
        # For regular DataFrame
        for col in returns.columns:
            sns.kdeplot(returns[col], label=col)
    
    plt.axvline(x=0, color='black', linestyle='--', alpha=0.5)
    plt.title(title)
    plt.xlabel("Return")
    plt.ylabel("Density")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

# Plot return distributions
plot_return_distributions(market_returns, f"{returns_method.capitalize()} Return Distributions")

## 7. Create Portfolio

In [ ]:
# Load portfolio weights from configuration
try:
    with open(CONFIG_DIR / "data_config.json", 'r') as f:
        data_config = json.load(f)
    
    # Extract weights from equity and bond indices
    weights = {}
    
    for item in data_config["data_retrieval"]["equity_indices"]:
        weights[item["ticker"]] = item["weight"]
    
    for item in data_config["data_retrieval"]["bond_indices"]:
        weights[item["ticker"]] = item["weight"]
    
    print("Portfolio weights from configuration:")
    for asset, weight in weights.items():
        print(f"  - {asset}: {weight:.2f}")
    
except Exception as e:
    print(f"Error loading portfolio weights: {e}")
    print("Using equal weights for all assets")
    weights = None

In [ ]:
# Before creating portfolio
from src.data.retrieve_data import load_api_keys, get_yahoo_finance_data

# If market data doesn't contain expected tickers, fetch them
if not any(ticker in str(market_returns.columns) for ticker in ['SPX', 'US10YT=RR', '^GSPC', '^TNX']):
    print("Market price data not found. Attempting to retrieve market data...")
    try:
        # Get config data for tickers
        with open(CONFIG_DIR / "data_config.json", 'r') as f:
            data_config = json.load(f)
        
        config = data_config['data_retrieval']
        equity_tickers = [item['ticker'] for item in config['equity_indices']]
        bond_tickers = [item['ticker'] for item in config['bond_indices']]
        
        # Retrieve market data
        market_price_data = get_yahoo_finance_data(
            tickers=equity_tickers + bond_tickers,
            start_date=market_returns.index[0].strftime('%Y-%m-%d'),
            end_date=market_returns.index[-1].strftime('%Y-%m-%d')
        )
        
        # Calculate returns
        market_price_returns = calculate_returns(market_price_data, method=returns_method)
        
        # Now create portfolio from this data
        portfolio_returns = create_portfolio(market_price_returns, weights=weights)
    except Exception as e:
        print(f"Error retrieving market data: {e}")
        # Create dummy portfolio
        portfolio_returns = pd.Series(index=market_returns.index, data=np.zeros(len(market_returns)))
        print("Created dummy portfolio due to missing market data")
else:
    # Standard portfolio creation
    portfolio_returns = create_portfolio(market_returns, weights=weights)

In [ ]:
# Check if we already have a valid portfolio from previous cell
if 'portfolio_returns' in locals() and not (isinstance(portfolio_returns, pd.Series) and portfolio_returns.isna().all()):
    print("Using previously created portfolio...")
else:
    # Create portfolio
    print("Creating portfolio from asset returns...")
    portfolio_returns = create_portfolio(market_returns, weights=weights)

print(f"Portfolio returns shape: {portfolio_returns.shape}")

# Display portfolio statistics
print("\nPortfolio statistics:")
print(f"Mean return: {portfolio_returns.mean():.6f}")
print(f"Standard deviation: {portfolio_returns.std():.6f}")
print(f"Minimum return: {portfolio_returns.min():.6f}")
print(f"Maximum return: {portfolio_returns.max():.6f}")
print(f"Skewness: {portfolio_returns.skew():.4f}")
print(f"Kurtosis: {portfolio_returns.kurtosis():.4f}")
print(f"Sharpe ratio (assuming 0% risk-free rate): {portfolio_returns.mean() / portfolio_returns.std():.4f}")

In [ ]:
# Plot portfolio returns
plt.figure(figsize=(14, 8))

# Plot portfolio returns
plt.subplot(2, 1, 1)
portfolio_returns.plot()
plt.axhline(y=0, color='black', linestyle='--', alpha=0.5)
plt.title("Portfolio Returns Over Time")
plt.xlabel("Date")
plt.ylabel("Return")
plt.grid(True)

# Plot cumulative returns
plt.subplot(2, 1, 2)
cumulative_returns = (1 + portfolio_returns).cumprod()
cumulative_returns.plot()
plt.title("Cumulative Portfolio Returns")
plt.xlabel("Date")
plt.ylabel("Cumulative Return")
plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# Plot return distribution with normal distribution overlay
plt.figure(figsize=(12, 8))

# Plot histogram of returns
sns.histplot(portfolio_returns, kde=True, stat="density", label="Actual Returns")

# Plot normal distribution with same mean and std
x = np.linspace(portfolio_returns.min(), portfolio_returns.max(), 1000)
y = stats.norm.pdf(x, portfolio_returns.mean(), portfolio_returns.std())
plt.plot(x, y, 'r--', label="Normal Distribution")

plt.axvline(x=0, color='black', linestyle='--', alpha=0.5)
plt.title("Portfolio Return Distribution vs. Normal Distribution")
plt.xlabel("Return")
plt.ylabel("Density")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

# Q-Q plot to check normality
plt.figure(figsize=(10, 10))
stats.probplot(portfolio_returns, dist="norm", plot=plt)
plt.title("Q-Q Plot of Portfolio Returns")
plt.grid(True)
plt.tight_layout()
plt.show()

## 8. Analyze Asset Correlations

In [ ]:
# Calculate and visualize asset correlations
def calculate_correlations(returns):
    if returns.empty:
        print("Empty DataFrame, skipping correlation analysis")
        return None
    
    # Handle MultiIndex DataFrame
    if isinstance(returns.columns, pd.MultiIndex):
        assets = returns.columns.levels[1]
        corr_data = pd.DataFrame(index=assets, columns=assets)
        
        for asset1 in assets:
            for asset2 in assets:
                # Extract returns for the assets
                asset1_returns = returns.xs(asset1, axis=1, level=1).iloc[:, 0]
                asset2_returns = returns.xs(asset2, axis=1, level=1).iloc[:, 0]
                
                # Calculate correlation
                correlation = asset1_returns.corr(asset2_returns)
                corr_data.loc[asset1, asset2] = correlation
    else:
        # For regular DataFrame
        corr_data = returns.corr()
    
    return corr_data

# Calculate correlations
correlations = calculate_correlations(market_returns)

if correlations is not None:
    # Display correlation matrix
    print("Asset Correlation Matrix:")
    display(correlations)
    
    # Visualize correlation matrix
    plt.figure(figsize=(12, 10))
    sns.heatmap(correlations, annot=True, cmap='coolwarm', vmin=-1, vmax=1, center=0, fmt='.2f')
    plt.title("Asset Return Correlations")
    plt.tight_layout()
    plt.show()

## 9. Analyze Relationship with Macroeconomic Factors

In [ ]:
# Analyze relationship between portfolio returns and macro factors
if not aligned_macro.empty and not portfolio_returns.empty:
    # Align dates
    common_dates = portfolio_returns.index.intersection(aligned_macro.index)
    aligned_portfolio = portfolio_returns.loc[common_dates]
    macro_subset = aligned_macro.loc[common_dates]
    
    if not common_dates.empty:
        print(f"Analyzing relationship with {len(macro_subset.columns)} macro factors over {len(common_dates)} dates")
        
        # Calculate correlations
        correlations = pd.DataFrame(index=macro_subset.columns, columns=['Correlation'])
        
        for factor in macro_subset.columns:
            correlations.loc[factor, 'Correlation'] = aligned_portfolio.corr(macro_subset[factor])
        
        # Sort by absolute correlation
        correlations['Abs_Correlation'] = correlations['Correlation'].abs()
        correlations = correlations.sort_values(by='Abs_Correlation', ascending=False)
        correlations = correlations.drop('Abs_Correlation', axis=1)
        
        # Display correlations
        print("\nPortfolio Return Correlations with Macro Factors:")
        display(correlations)
        
        # Plot top correlations
        plt.figure(figsize=(10, 6))
        correlations['Correlation'].plot(kind='bar')
        plt.title("Portfolio Return Correlations with Macro Factors")
        plt.ylabel("Correlation")
        plt.axhline(y=0, color='black', linestyle='--', alpha=0.5)
        plt.grid(True)
        plt.tight_layout()
        plt.show()
        
        # Plot scatter plots for top factors
        top_factors = correlations.index[:min(3, len(correlations))]
        
        plt.figure(figsize=(15, 5))
        for i, factor in enumerate(top_factors):
            plt.subplot(1, 3, i+1)
            plt.scatter(macro_subset[factor], aligned_portfolio, alpha=0.5)
            plt.title(f"{factor} vs. Portfolio Returns")
            plt.xlabel(factor)
            plt.ylabel("Portfolio Return")
            plt.grid(True)
            
            # Add regression line
            z = np.polyfit(macro_subset[factor], aligned_portfolio, 1)
            p = np.poly1d(z)
            plt.plot(macro_subset[factor], p(macro_subset[factor]), "r--")
        
        plt.tight_layout()
        plt.show()
    else:
        print("No common dates between portfolio returns and macro factors")
else:
    print("Skipping macro factor analysis (no macro data available)")

## 10. Save Processed Data for Risk Models

In [ ]:
# Save the processed data for use in risk models
def save_data(df, filename, directory=PROCESSED_DIR):
    if df.empty:
        print(f"Empty DataFrame, skipping save for {filename}")
        return False
    
    # Make sure directory exists
    directory.mkdir(exist_ok=True, parents=True)
    
    # Save to CSV
    file_path = directory / filename
    df.to_csv(file_path)
    print(f"Saved {len(df)} rows to {file_path}")
    return True

# Save all processed data
print("Saving processed data for risk models...")
save_data(aligned_market, "market_data.csv")
save_data(aligned_macro, "macro_data.csv")
save_data(market_returns, "market_returns.csv")
save_data(portfolio_returns, "portfolio_returns.csv")
save_data(correlations, "asset_correlations.csv")

## Summary and Next Steps

In this notebook, we've performed comprehensive data cleaning and preprocessing for our risk management dashboard:

1. Loaded and inspected raw market and macroeconomic data
2. Identified and addressed data quality issues (missing values, outliers)
3. Cleaned and aligned datasets to common dates
4. Calculated returns using the configured method
5. Created a portfolio based on asset weights
6. Analyzed return distributions and correlations
7. Examined relationships with macroeconomic factors
8. Saved processed data for use in risk models

### Key Findings:

- [Note: Add your key findings here based on the actual analysis results]

### Next Steps:

1. Proceed to VaR modeling (`03_var_modeling.ipynb`)
2. Implement Monte Carlo simulations (`04_monte_carlo_sim.ipynb`)
3. Develop stress testing scenarios (`05_stress_testing.ipynb`)
4. Validate models through backtesting (`06_backtesting.ipynb`)